# Advanced Querying Mongo

In [213]:
import pandas as pd
import numpy as np
import pymongo
pd.options.display.max_columns = None

Importing libraries and setting up connection

In [214]:
cliente = pymongo.MongoClient()
cliente

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [215]:
cliente.list_database_names()

['admin',
 'config',
 'lab_mongo',
 'local',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata']

In [216]:
mydb = cliente["lab_mongo"]
coleccion = mydb.companies

In [217]:
pd.DataFrame(coleccion.find()).columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots',
       'external_links', 'partners', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'ipo'],
      dtype='object')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [218]:
list(coleccion.find(({"name":"Babelgum"}), {"name":1,"_id":0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [242]:
employee_size = pd.DataFrame((coleccion.find(({"number_of_employees":{"$gt": 5000}}), {"name":1, "_id":0, "number_of_employees":1}).sort("number_of_employees", -1).limit(20)))
index = pd.Index(range(1, 21))
employee_size = employee_size.set_index(index)
employee_size.head(5)

,name,number_of_employees
1,Siemens,405000
2,IBM,388000
3,Toyota,320000
4,PayPal,300000
5,Nippon Telegraph and Telephone Corporation,227000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [220]:
cond1 = {"founded_year":{"$gte": 2000}}
cond2 = {"founded_year": {"$lte": 2005}}

list(coleccion.find({'$and': [cond1, cond2]}, {"name":1,"_id":0,"founded_year":1}).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [221]:
cond3 = {"ipo.valuation_amount":{"$gte": 100000000}}
cond4 = {"founded_year": {"$lte": 2010}}

list(coleccion.find({'$and': [cond3, cond4]}, {"name":1,"_id":0,"ipo.valuation_amount":1}).sort("ipo.valuation_amount",-1).limit(5))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [222]:
cond5 = {"number_of_employees":{"$lte": 1000}}
cond6 = {"founded_year": {"$lte": 2005}}

list(coleccion.find({'$and': [cond5, cond6]}, {"name":1,"_id":0,"number_of_employees":1}).sort("number_of_employees", -1).limit(10))

[{'name': 'Omniture', 'number_of_employees': 1000},
 {'name': 'Telenav', 'number_of_employees': 1000},
 {'name': 'Gumtree', 'number_of_employees': 1000},
 {'name': 'SK Net Service Company Ltd', 'number_of_employees': 1000},
 {'name': 'Yodle', 'number_of_employees': 1000},
 {'name': "OC'LIANE", 'number_of_employees': 1000},
 {'name': 'FANUC Robotics America', 'number_of_employees': 1000},
 {'name': 'Antal International', 'number_of_employees': 1000},
 {'name': 'Sonus Networks', 'number_of_employees': 1000},
 {'name': 'Akamai Technologies', 'number_of_employees': 1000}]

### 6. All the companies that don't include the `partners` field.

In [223]:
sin_partner = list(coleccion.find({'partners':{'$exists':False}}, {"name":1,"_id":0})) #deberia funcionar, sin embargo los campos partners existen en todos los registros pero con listas vacias
print(sin_partner)
sin_partner = list(coleccion.find({'partners':{'$size':0}}, {"name":1, "partners":1, "_id":0}).limit(5)) #muestra un ejemplo de las compañias que no tienen partner y deberian estar incluidas en la lista anterior vacia
sin_partner

[]


[{'name': 'Wetpaint', 'partners': []},
 {'name': 'AdventNet', 'partners': []},
 {'name': 'Zoho', 'partners': []},
 {'name': 'Digg', 'partners': []},
 {'name': 'Facebook', 'partners': []}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [224]:
list(coleccion.find({"category_code": {"$type": "null"}}, {"_id":0,"category_code":1, "name":1}).limit(5))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [225]:
cond7 = {"number_of_employees":{"$gte": 100}}
cond8 = {"number_of_employees":{"$lt": 1000}}

list(coleccion.find({'$and': [cond7, cond8]}, {"name":1,"_id":0,"number_of_employees":1}).sort("number_of_employees", -1).limit(5))

[{'name': 'Datamonitor', 'number_of_employees': 984},
 {'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931}]

### 9. Order all the companies by their IPO price in a descending order.

In [226]:
ipo_price = pd.DataFrame((coleccion.find({"ipo.valuation_amount":{"$gte":1}},{"name":1,"_id":0,"ipo.valuation_amount":1}).sort("ipo.valuation_amount", -1)))
ipo_price.drop_duplicates(subset=['name']).head(5)

,name,ipo
0,GREE,{'valuation_amount': 108960000000}
1,Facebook,{'valuation_amount': 104000000000}
2,Amazon,{'valuation_amount': 100000000000}
3,Twitter,{'valuation_amount': 18100000000}
4,Groupon,{'valuation_amount': 12800000000}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [227]:
list(coleccion.find({}, {"name":1,"_id":0,"number_of_employees":1}).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [228]:
list(coleccion.find({"founded_month":{"$gt":6}}, {"name":1,"_id":0,"founded_month":1}).limit(5))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [229]:
cond9 = {"founded_year":{"$lt": 2000}}
cond10 = {"acquisition.price_amount":{"$gt": 1000000}}

pd.DataFrame(coleccion.find({'$and': [cond9, cond10]}, {"name":1,"_id":0,"founded_year":1, "acquisition.price_amount":1}).sort("acquisition.price_amount", -1).limit(8))

,name,founded_year,acquisition
0,BEA Systems,1995,{'price_amount': 8500000000}
1,Navteq,1985,{'price_amount': 8100000000}
2,Sun Microsystems,1982,{'price_amount': 7400000000}
3,Pixar,1986,{'price_amount': 7400000000}
4,LSI,1980,{'price_amount': 6600000000}
5,National Semiconductor,1959,{'price_amount': 6500000000}
6,aQuantive,1997,{'price_amount': 6400000000}
7,Siebel Systems,1993,{'price_amount': 5850000000}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [230]:
pd.DataFrame(coleccion.find({"acquisition.acquired_year":{"$gt":2010}}, 
                            {"name":1,
                             "_id":0,
                             "acquisition":1}).sort("acquisition.price_amount", -1).limit(5))

,name,acquisition
0,T-Mobile,"{'price_amount': 39000000000, 'price_currency_..."
1,Goodrich Corporation,"{'price_amount': 18400000000, 'price_currency_..."
2,LSI,"{'price_amount': 6600000000, 'price_currency_c..."
3,National Semiconductor,"{'price_amount': 6500000000, 'price_currency_c..."
4,Ariba,"{'price_amount': 4300000000, 'price_currency_c..."


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [231]:
list(coleccion.find({"founded_year":{"$type":"number"}}, {"name":1, "_id":0, "founded_year":1}).sort("founded_year", 1).limit(5))

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [232]:
list(coleccion.find({"founded_day":{"$lte":7}}, {"acquisition.price_amount":1, "name":1, "_id":0}).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [233]:
list(coleccion.find({"category_code":"web"}, {"name":1, "category_code":1, "_id":0}).sort("number_of_employees", 1).limit(5))

[{'name': 'StumbleUpon', 'category_code': 'web'},
 {'name': 'Gannett', 'category_code': 'web'},
 {'name': 'Viacom', 'category_code': 'web'},
 {'name': 'Gizmoz', 'category_code': 'web'},
 {'name': 'Postini', 'category_code': 'web'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [234]:
cond11 = {"acquisition.price_amount":{"$gt": 10000000}}
cond12 = {"acquisition.price_currency_code":"EUR"}

list(coleccion.find({"$and":[cond11, cond12]}, {"name":1, "_id":0}))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [235]:
list(coleccion.find({"acquisition.acquired_month":{"$lt":4}}, {"name":1, "_id":0, "acquisition":1}).limit(1))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [236]:
cond13 = {"founded_year":{"$gte":2000}}
cond14 = {"founded_year":{"$lt":2010}}
cond15 = {"acquisition.acquired_year":{"$gte":2011}}

pd.DataFrame(coleccion.find({'$and': [cond13, cond14, cond15]}, {"name":1,"_id":0, "founded_year":1, "acquisition.acquired_year":1}).limit(5))

,name,founded_year,acquisition
0,Wetpaint,2005,{'acquired_year': 2013}
1,Digg,2004,{'acquired_year': 2012}
2,Geni,2006,{'acquired_year': 2012}
3,Kyte,2006,{'acquired_year': 2011}
4,Jingle Networks,2005,{'acquired_year': 2011}


### 20. All the companies that have been 'deadpooled' after the third year.

In [239]:
df_deadpooled = pd.DataFrame(coleccion.find({"deadpooled_year":{"$gt":0}},{"name":1, "_id":0, "deadpooled_year":1, "founded_year":1}))
df_deadpooled["creation_deadpool_years"] = df_deadpooled["deadpooled_year"] - df_deadpooled["founded_year"]
df_deadpooled[df_deadpooled["creation_deadpool_years"] > 3].sample(6)

,name,founded_year,deadpooled_year,creation_deadpool_years
93,Mixaloo,2007.0,2013,6.0
593,pickrset,2007.0,2012,5.0
438,Sequoia Communications,2000.0,2011,11.0
848,Vocab,2002.0,2012,10.0
283,Dilithium Networks,2001.0,2012,11.0
661,ChipVision Design,2002.0,2011,9.0
